# IF3070 Foundations of Artificial Intelligence | Tugas Besar 2

This notebook serves as a template for the assignment. Please create a copy of this notebook to complete your work. You can add more code blocks, markdown blocks, or new sections if needed.


Group Number: 13

Group Members:
- Jonathan Wiguna (18222019)
- Naomi Pricilla Agustine(18222065)
- Harry Truman Salim (18222081)
- Micky Valentino(18222093)

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

## Import Dataset

In [2]:
df = pd.read_csv('https://drive.google.com/uc?id=1KJsJrbObEun6NRme9jMIjX8SyIeuTY5M')
df.head()

,id,FILENAME,URL,URLLength,Domain,DomainLength,IsDomainIP,TLD,CharContinuationRate,TLDLegitimateProb,...,Pay,Crypto,HasCopyrightInfo,NoOfImage,NoOfCSS,NoOfJS,NoOfSelfRef,NoOfEmptyRef,NoOfExternalRef,label
0,1,NaN,https://www.northcm.ac.th,24.0,www.northcm.ac.th,17.0,0.0,NaN,0.800000,NaN,...,0.0,0.0,1.0,NaN,3.0,NaN,69.0,NaN,NaN,1
1,4,8135291.txt,http://uqr.to/1il1z,NaN,NaN,NaN,NaN,to,1.000000,0.000896,...,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1.0,0
2,5,586561.txt,https://www.woolworthsrewards.com.au,35.0,www.woolworthsrewards.com.au,28.0,0.0,au,0.857143,NaN,...,1.0,0.0,1.0,33.0,7.0,8.0,15.0,NaN,2.0,1
3,6,NaN,NaN,31.0,NaN,NaN,NaN,com,0.562500,0.522907,...,1.0,0.0,1.0,24.0,5.0,14.0,NaN,NaN,NaN,1
4,11,412632.txt,NaN,NaN,www.nyprowrestling.com,22.0,0.0,NaN,1.000000,NaN,...,0.0,0.0,1.0,NaN,NaN,14.0,NaN,0.0,NaN,1


In [3]:
df.describe()

,id,URLLength,DomainLength,IsDomainIP,CharContinuationRate,TLDLegitimateProb,URLCharProb,TLDLength,NoOfSubDomain,HasObfuscation,...,Pay,Crypto,HasCopyrightInfo,NoOfImage,NoOfCSS,NoOfJS,NoOfSelfRef,NoOfEmptyRef,NoOfExternalRef,label
count,140404.000000,79765.000000,94085.000000,98274.000000,92362.000000,87531.000000,88333.000000,92673.000000,96344.000000,74684.000000,...,97230.000000,90207.000000,73059.000000,89932.000000,73270.000000,79603.000000,92272.000000,97718.000000,71025.000000,140404.000000
mean,117682.632746,27.701473,19.619387,0.000488,0.918007,0.277482,0.059286,2.729015,1.161442,0.000402,...,0.345706,0.034454,0.751543,41.647489,10.268623,16.461729,104.875900,3.733846,78.104273,0.924831
std,68122.005080,23.140676,5.833808,0.022095,0.159142,0.248115,0.008063,0.533310,0.445054,0.020038,...,0.475600,0.182393,0.432121,102.538702,133.321659,21.001274,219.137035,24.192458,187.106965,0.263664
min,1.000000,14.000000,4.000000,0.000000,0.000000,0.000000,0.001229,2.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,58689.500000,23.000000,16.000000,0.000000,0.913043,0.012927,0.055542,2.000000,1.000000,0.000000,...,0.000000,0.000000,1.000000,10.000000,2.000000,6.000000,24.000000,0.000000,13.000000,1.000000
50%,117421.500000,26.000000,19.000000,0.000000,1.000000,0.522907,0.060264,3.000000,1.000000,0.000000,...,0.000000,0.000000,1.000000,23.000000,5.000000,12.000000,69.000000,0.000000,39.000000,1.000000
75%,176724.250000,30.000000,23.000000,0.000000,1.000000,0.522907,0.064391,3.000000,1.000000,0.000000,...,1.000000,0.000000,1.000000,45.000000,12.000000,22.000000,132.000000,3.000000,99.000000,1.000000
max,235795.000000,4054.000000,93.000000,1.000000,1.000000,0.522907,0.088766,13.000000,7.000000,1.000000,...,1.000000,1.000000,1.000000,8956.000000,35820.000000,2828.000000,26596.000000,4887.000000,27516.000000,1.000000


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140404 entries, 0 to 140403
Data columns (total 56 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   id                          140404 non-null  int64  
 1   FILENAME                    82872 non-null   object 
 2   URL                         96917 non-null   object 
 3   URLLength                   79765 non-null   float64
 4   Domain                      70207 non-null   object 
 5   DomainLength                94085 non-null   float64
 6   IsDomainIP                  98274 non-null   float64
 7   TLD                         95005 non-null   object 
 8   CharContinuationRate        92362 non-null   float64
 9   TLDLegitimateProb           87531 non-null   float64
 10  URLCharProb                 88333 non-null   float64
 11  TLDLength                   92673 non-null   float64
 12  NoOfSubDomain               96344 non-null   float64
 13  HasObfuscation

In [5]:
df = df.drop(columns=['FILENAME'])


In [6]:
# pemisahan fitur yang bertipe kategori dan non-kategori agar dianalisis dengan pendekatan yang berbeda
categorical = ['TLD',  'Title', 'Domain','URL',  'IsDomainIP', 'HasObfuscation', 'IsHTTPS', 'HasTitle', 'HasFavicon', 'Robots', 'IsResponsive', 'HasDescription', 'HasExternalFormSubmit', 'HasSocialNet', 'HasSubmitButton', 'HasHiddenFields', 'HasPasswordField', 'Bank', 'Pay', 'Crypto', 'HasCopyrightInfo']

non_categorical = [col for col in df.columns if col not in categorical]

# 1. Split Training Set and Validation Set

Splitting the training and validation set works as an early diagnostic towards the performance of the model we train. This is done before the preprocessing steps to **avoid data leakage inbetween the sets**. If you want to use k-fold cross-validation, split the data later and do the cleaning and preprocessing separately for each split.

Note: For training, you should use the data contained in the `train` folder given by the TA. The `test` data is only used for kaggle submission.

In [7]:
# Split training set and validation set here, store into variables train_set and val_set.
# Remember to also keep the original training set before splitting. This will come important later.
# train_set, val_set = ...
train_set, val_set = train_test_split(df, test_size=0.3, random_state=42)

In [8]:
train_set

,id,URL,URLLength,Domain,DomainLength,IsDomainIP,TLD,CharContinuationRate,TLDLegitimateProb,URLCharProb,...,Pay,Crypto,HasCopyrightInfo,NoOfImage,NoOfCSS,NoOfJS,NoOfSelfRef,NoOfEmptyRef,NoOfExternalRef,label
95455,160156,https://www.politis.fr,NaN,www.politis.fr,NaN,NaN,fr,1.00,NaN,0.052775,...,0.0,0.0,NaN,NaN,NaN,NaN,243.0,NaN,NaN,1
22009,36650,https://www.perrycountylibrary.org,NaN,NaN,NaN,0.0,org,1.00,0.079963,NaN,...,NaN,0.0,NaN,3.0,21.0,35.0,56.0,2.0,65.0,1
96744,162391,https://www.seino.co.jp,22.0,www.seino.co.jp,NaN,0.0,NaN,0.75,NaN,0.068660,...,0.0,0.0,NaN,NaN,NaN,NaN,373.0,1.0,NaN,1
112542,188962,https://www.cunninglinguist.co,NaN,NaN,22.0,0.0,co,1.00,0.005977,NaN,...,1.0,0.0,NaN,NaN,3.0,NaN,NaN,0.0,NaN,1
35264,58949,https://www.profitautomationcoach.com,NaN,NaN,29.0,NaN,com,1.00,0.522907,NaN,...,0.0,NaN,1.0,12.0,NaN,NaN,NaN,NaN,83.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110268,185151,https://www.gemathis.com,23.0,www.gemathis.com,16.0,0.0,com,1.00,0.522907,NaN,...,NaN,0.0,1.0,NaN,2.0,NaN,106.0,7.0,NaN,1
119879,201338,NaN,24.0,www.mural24.co.uk,NaN,0.0,uk,0.80,NaN,NaN,...,1.0,0.0,NaN,166.0,3.0,7.0,NaN,1.0,NaN,1
103694,174060,NaN,34.0,NaN,27.0,0.0,NaN,1.00,NaN,NaN,...,1.0,0.0,NaN,NaN,NaN,NaN,42.0,NaN,25.0,1
131932,221529,https://www.leinsterrugby.ie,27.0,NaN,20.0,0.0,ie,NaN,0.001588,0.052675,...,0.0,0.0,1.0,NaN,NaN,57.0,NaN,0.0,331.0,1


In [9]:
val_set

,id,URL,URLLength,Domain,DomainLength,IsDomainIP,TLD,CharContinuationRate,TLDLegitimateProb,URLCharProb,...,Pay,Crypto,HasCopyrightInfo,NoOfImage,NoOfCSS,NoOfJS,NoOfSelfRef,NoOfEmptyRef,NoOfExternalRef,label
8409,14051,https://www.citroencarclub.org.uk,NaN,NaN,25.0,0.0,uk,NaN,0.028555,0.061899,...,0.0,0.0,1.0,NaN,20.0,27.0,2.0,14.0,NaN,1
20959,34872,https://www.flipyourtext.com,NaN,www.flipyourtext.com,NaN,0.0,com,NaN,0.522907,NaN,...,NaN,0.0,NaN,2.0,NaN,9.0,8.0,0.0,NaN,1
88797,148915,NaN,25.0,NaN,18.0,0.0,NaN,NaN,0.522907,NaN,...,1.0,NaN,1.0,3.0,3.0,NaN,2.0,NaN,2.0,1
24345,40612,https://www.elcalafate.gov.ar,28.0,NaN,NaN,NaN,NaN,NaN,NaN,0.062508,...,0.0,NaN,1.0,NaN,NaN,40.0,82.0,2.0,NaN,1
95019,159405,https://www.dongwon.com,22.0,NaN,NaN,0.0,NaN,1.000000,0.522907,0.062712,...,0.0,0.0,1.0,NaN,12.0,9.0,63.0,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86243,144551,https://www.skiurlaub-infos.com,30.0,NaN,23.0,0.0,NaN,NaN,0.522907,0.054795,...,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0,54.0,1
100637,168914,https://sbi-n78b8.web.app/,25.0,sbi-n78b8.web.app,17.0,0.0,app,0.461538,NaN,NaN,...,0.0,NaN,NaN,0.0,0.0,0.0,NaN,0.0,3.0,0
45620,76303,NaN,NaN,NaN,22.0,NaN,eg,0.800000,0.000106,0.065158,...,0.0,0.0,NaN,53.0,NaN,51.0,169.0,54.0,221.0,1
131318,220493,https://www.tattoo-bewertung.de,NaN,www.tattoo-bewertung.de,23.0,NaN,NaN,NaN,0.032650,NaN,...,0.0,NaN,NaN,14.0,NaN,7.0,86.0,NaN,NaN,1


# 2. Data Cleaning and Preprocessing

This step is the first thing to be done once a Data Scientist have grasped a general knowledge of the data. Raw data is **seldom ready for training**, therefore steps need to be taken to clean and format the data for the Machine Learning model to interpret.

By performing data cleaning and preprocessing, you ensure that your dataset is ready for model training, leading to more accurate and reliable machine learning results. These steps are essential for transforming raw data into a format that machine learning algorithms can effectively learn from and make predictions.

We will give some common methods for you to try, but you only have to **at least implement one method for each process**. For each step that you will do, **please explain the reason why did you do that process. Write it in a markdown cell under the code cell you wrote.**

## A. Data Cleaning

**Data cleaning** is the crucial first step in preparing your dataset for machine learning. Raw data collected from various sources is often messy and may contain errors, missing values, and inconsistencies. Data cleaning involves the following steps:

1. **Handling Missing Data:** Identify and address missing values in the dataset. This can include imputing missing values, removing rows or columns with excessive missing data, or using more advanced techniques like interpolation.

2. **Dealing with Outliers:** Identify and handle outliers, which are data points significantly different from the rest of the dataset. Outliers can be removed or transformed to improve model performance.

3. **Data Validation:** Check for data integrity and consistency. Ensure that data types are correct, categorical variables have consistent labels, and numerical values fall within expected ranges.

4. **Removing Duplicates:** Identify and remove duplicate rows, as they can skew the model's training process and evaluation metrics.

5. **Feature Engineering**: Create new features or modify existing ones to extract relevant information. This step can involve scaling, normalizing, or encoding features for better model interpretability.

### I. Handling Missing Data

Missing data can adversely affect the performance and accuracy of machine learning models. There are several strategies to handle missing data in machine learning:

1. **Data Imputation:**

    a. **Mean, Median, or Mode Imputation:** For numerical features, you can replace missing values with the mean, median, or mode of the non-missing values in the same feature. This method is simple and often effective when data is missing at random.

    b. **Constant Value Imputation:** You can replace missing values with a predefined constant value (e.g., 0) if it makes sense for your dataset and problem.

    c. **Imputation Using Predictive Models:** More advanced techniques involve using predictive models to estimate missing values. For example, you can train a regression model to predict missing numerical values or a classification model to predict missing categorical values.

2. **Deletion of Missing Data:**

    a. **Listwise Deletion:** In cases where the amount of missing data is relatively small, you can simply remove rows with missing values from your dataset. However, this approach can lead to a loss of valuable information.

    b. **Column (Feature) Deletion:** If a feature has a large number of missing values and is not critical for your analysis, you can consider removing that feature altogether.

3. **Domain-Specific Strategies:**

    a. **Domain Knowledge:** In some cases, domain knowledge can guide the imputation process. For example, if you know that missing values are related to a specific condition, you can impute them accordingly.

4. **Imputation Libraries:**

    a. **Scikit-Learn:** Scikit-Learn provides a `SimpleImputer` class that can handle basic imputation strategies like mean, median, and mode imputation.

    b. **Fancyimpute:** Fancyimpute is a Python library that offers more advanced imputation techniques, including matrix factorization, k-nearest neighbors, and deep learning-based methods.

The choice of imputation method should be guided by the nature of your data, the amount of missing data, the problem you are trying to solve, and the assumptions you are willing to make.

Identify Missing Values

In [10]:
# Identifikasi missing values
print("Jumlah missing values:")
print(df.isnull().sum())
print()

# Menghitung persentase missing values
missing_percent = (df.isnull().sum() / len(df)) * 100
print("Persentase missing values:")
print(missing_percent)

Jumlah missing values:
id                                0
URL                           43487
URLLength                     60639
Domain                        70197
DomainLength                  46319
IsDomainIP                    42130
TLD                           45399
CharContinuationRate          48042
TLDLegitimateProb             52873
URLCharProb                   52071
TLDLength                     47731
NoOfSubDomain                 44060
HasObfuscation                65720
NoOfObfuscatedChar            66798
ObfuscationRatio              64598
NoOfLettersInURL              63338
LetterRatioInURL              65746
NoOfDegitsInURL               58810
DegitRatioInURL               53508
NoOfEqualsInURL               61578
NoOfQMarkInURL                44101
NoOfAmpersandInURL            45387
NoOfOtherSpecialCharsInURL    47629
SpacialCharRatioInURL         62834
IsHTTPS                       49362
LineOfCode                    69153
LargestLineLength             67928
HasTi

Data Imputation

In [11]:
# Pisahkan kolom berdasarkan fungsinya
boolean_cols = ['IsDomainIP', 'HasObfuscation', 'IsHTTPS', 'HasTitle',
                'HasFavicon', 'IsResponsive', 'HasDescription',
                'HasExternalFormSubmit', 'HasSocialNet', 'HasSubmitButton',
                'HasHiddenFields', 'HasPasswordField', 'HasCopyrightInfo']

count_cols = ['NoOfSubDomain', 'NoOfObfuscatedChar', 'NoOfLettersInURL',
              'NoOfDegitsInURL', 'NoOfEqualsInURL', 'NoOfQMarkInURL',
              'NoOfAmpersandInURL', 'NoOfOtherSpecialCharsInURL',
              'NoOfURLRedirect', 'NoOfSelfRedirect', 'NoOfPopup',
              'NoOfiFrame', 'NoOfImage', 'NoOfCSS', 'NoOfJS',
              'NoOfSelfRef', 'NoOfEmptyRef']

# Tangani URL dan Domain length
if 'URL' in df.columns and 'URLLength' in df.columns:
    df.loc[df['URL'].isna(), 'URLLength'] = 0
    df.loc[df['URL'].notna(), 'URLLength'] = df.loc[df['URL'].notna(), 'URL'].str.len()

if 'Domain' in df.columns and 'DomainLength' in df.columns:
    df.loc[df['Domain'].isna(), 'DomainLength'] = 0
    df.loc[df['Domain'].notna(), 'DomainLength'] = df.loc[df['Domain'].notna(), 'Domain'].str.len()

# Isi nilai 0 untuk kolom boolean dan count
df[boolean_cols] = df[boolean_cols].fillna(0)
df[count_cols] = df[count_cols].fillna(0)

# Tangani semua kolom numerik tersisa dengan median
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
numeric_cols = [col for col in numeric_cols if col not in boolean_cols + count_cols]
numeric_imputer = SimpleImputer(strategy='median')
df[numeric_cols] = numeric_imputer.fit_transform(df[numeric_cols])

# Tangani semua kolom kategorikal
categorical_cols = df.select_dtypes(include=['object']).columns
categorical_imputer = SimpleImputer(strategy='most_frequent')
df[categorical_cols] = categorical_imputer.fit_transform(df[categorical_cols])

# Cek hasil
print("\nJumlah missing values setelah imputasi:")
print(df.isnull().sum())


Jumlah missing values setelah imputasi:
id                            0
URL                           0
URLLength                     0
Domain                        0
DomainLength                  0
IsDomainIP                    0
TLD                           0
CharContinuationRate          0
TLDLegitimateProb             0
URLCharProb                   0
TLDLength                     0
NoOfSubDomain                 0
HasObfuscation                0
NoOfObfuscatedChar            0
ObfuscationRatio              0
NoOfLettersInURL              0
LetterRatioInURL              0
NoOfDegitsInURL               0
DegitRatioInURL               0
NoOfEqualsInURL               0
NoOfQMarkInURL                0
NoOfAmpersandInURL            0
NoOfOtherSpecialCharsInURL    0
SpacialCharRatioInURL         0
IsHTTPS                       0
LineOfCode                    0
LargestLineLength             0
HasTitle                      0
Title                         0
DomainTitleMatchScore         0

In [12]:
df.describe()

,id,URLLength,DomainLength,IsDomainIP,CharContinuationRate,TLDLegitimateProb,URLCharProb,TLDLength,NoOfSubDomain,HasObfuscation,...,Pay,Crypto,HasCopyrightInfo,NoOfImage,NoOfCSS,NoOfJS,NoOfSelfRef,NoOfEmptyRef,NoOfExternalRef,label
count,140404.000000,140404.000000,140404.000000,140404.000000,140404.000000,140404.000000,140404.000000,140404.000000,140404.000000,140404.000000,...,140404.000000,140404.000000,140404.000000,140404.000000,140404.000000,140404.000000,140404.000000,140404.000000,140404.000000,140404.000000
mean,117682.632746,19.778390,9.806914,0.000342,0.946063,0.369904,0.059649,2.821138,0.796972,0.000214,...,0.239402,0.022136,0.391064,26.676177,5.358693,9.333089,68.923314,2.598672,58.781352,0.924831
std,68122.005080,22.158927,10.654606,0.018487,0.134809,0.229171,0.006413,0.451892,0.652986,0.014616,...,0.426720,0.147127,0.487990,84.462605,96.446779,17.792966,184.490605,20.255539,134.505881,0.263664
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001229,2.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,58689.500000,0.000000,0.000000,0.000000,1.000000,0.079963,0.058559,3.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,38.000000,1.000000
50%,117421.500000,24.000000,4.000000,0.000000,1.000000,0.522907,0.060264,3.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,9.000000,0.000000,2.000000,22.000000,0.000000,39.000000,1.000000
75%,176724.250000,29.000000,19.000000,0.000000,1.000000,0.522907,0.061889,3.000000,1.000000,0.000000,...,0.000000,0.000000,1.000000,30.000000,6.000000,14.000000,94.000000,1.000000,40.000000,1.000000
max,235795.000000,4054.000000,110.000000,1.000000,1.000000,0.522907,0.088766,13.000000,7.000000,1.000000,...,1.000000,1.000000,1.000000,8956.000000,35820.000000,2828.000000,26596.000000,4887.000000,27516.000000,1.000000


### II. Dealing with Outliers

Outliers are data points that significantly differ from the majority of the data. They can be unusually high or low values that do not fit the pattern of the rest of the dataset. Outliers can significantly impact model performance, so it is important to handle them properly.

Some methods to handle outliers:
1. **Imputation**: Replace with mean, median, or a boundary value.
2. **Clipping**: Cap values to upper and lower limits.
3. **Transformation**: Use log, square root, or power transformations to reduce their influence.
4. **Model-Based**: Use algorithms robust to outliers (e.g., tree-based models, Huber regression).

In [13]:
import pandas as pd
import numpy as np

# Misalkan val_set adalah DataFrame Anda dan non_categorical adalah daftar kolom non-kategorikal
threshold_percentage = 0.03  # Batas maksimal 3% outlier

outlier = True
while outlier:
    change = False
    for col in non_categorical:
        if val_set[col].skew() > 1 or val_set[col].skew() < -1:
            # Menggunakan IQR untuk mendeteksi outlier
            Q1 = val_set[col].quantile(0.25)
            Q3 = val_set[col].quantile(0.75)
            IQR = Q3 - Q1
            # Mendapatkan indeks outlier
            outlier_indices = val_set[(val_set[col] < Q1 - 1.5 * IQR) | (val_set[col] > Q3 + 1.5 * IQR)].index
        else:
            # Menggunakan z-score untuk mendeteksi outlier
            z_scores = np.abs((val_set[col] - val_set[col].mean()) / val_set[col].std())
            outlier_indices = val_set[z_scores > 3].index
        
        # Hitung persentase outlier
        outlier_percentage = len(outlier_indices) / len(val_set)
        if 0 < outlier_percentage < threshold_percentage:
            val_set = val_set.drop(outlier_indices)
            change = True
        elif outlier_percentage > threshold_percentage:
            Q1 = val_set[col].quantile(0.25)
            Q3 = val_set[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR

            val_set[col] = np.where(val_set[col] < lower_bound, lower_bound, val_set[col])
            val_set[col] = np.where(val_set[col] > upper_bound, upper_bound, val_set[col])

    print (change)
    if not(change):
        outlier = False

    # Reset index setelah penghapusan outlier
    val_set.reset_index(drop=True, inplace=True)


True
True
False


In [14]:
import pandas as pd
import numpy as np

# Misalkan train_set adalah DataFrame Anda dan non_categorical adalah daftar kolom non-kategorikal
threshold_percentage = 0.03  # Batas maksimal 3% outlier
outlier = True
while outlier:
    change = False
    for col in non_categorical:
        if train_set[col].skew() > 1 or train_set[col].skew() < -1:
            # Menggunakan IQR untuk mendeteksi outlier
            Q1 = train_set[col].quantile(0.25)
            Q3 = train_set[col].quantile(0.75)
            IQR = Q3 - Q1
            # Mendapatkan indeks outlier
            outlier_indices = train_set[(train_set[col] < Q1 - 1.5 * IQR) | (train_set[col] > Q3 + 1.5 * IQR)].index
        else:
            # Menggunakan z-score untuk mendeteksi outlier
            z_scores = np.abs((train_set[col] - train_set[col].mean()) / train_set[col].std())
            outlier_indices = train_set[z_scores > 3].index
        
        # Hitung persentase outlier
        outlier_percentage = len(outlier_indices) / len(train_set)
        if 0 < outlier_percentage < threshold_percentage:
            train_set = train_set.drop(outlier_indices)
            change = True
        elif outlier_percentage > threshold_percentage:
            Q1 = train_set[col].quantile(0.25)
            Q3 = train_set[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR

            train_set[col] = np.where(train_set[col] < lower_bound, lower_bound, train_set[col])
            train_set[col] = np.where(train_set[col] > upper_bound, upper_bound, train_set[col])

    print (change)
    if not(change):
        outlier = False

    # Reset index setelah penghapusan outlier
    train_set.reset_index(drop=True, inplace=True)


True
True
False


In [15]:
for col in non_categorical:
    if train_set[col].skew() > 1 or train_set[col].skew() < -1:
        # Menggunakan IQR untuk mendeteksi outlier
        Q1 = train_set[col].quantile(0.25)
        Q3 = train_set[col].quantile(0.75)
        IQR = Q3 - Q1
        # Mendapatkan indeks outlier
        outlier_indices = train_set[(train_set[col] < Q1 - 1.5 * IQR) | (train_set[col] > Q3 + 1.5 * IQR)].index
    else:
        # Menggunakan z-score untuk mendeteksi outlier
        z_scores = np.abs((train_set[col] - train_set[col].mean()) / train_set[col].std())
        outlier_indices = train_set[z_scores > 3].index
    
    # Hitung persentase outlier
    outlier_percentage = len(outlier_indices) / len(train_set) * 100

    print (outlier_percentage)

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [16]:
for col in non_categorical:
    if val_set[col].skew() > 1 or val_set[col].skew() < -1:
        # Menggunakan IQR untuk mendeteksi outlier
        Q1 = val_set[col].quantile(0.25)
        Q3 = val_set[col].quantile(0.75)
        IQR = Q3 - Q1
        # Mendapatkan indeks outlier
        outlier_indices = val_set[(val_set[col] < Q1 - 1.5 * IQR) | (val_set[col] > Q3 + 1.5 * IQR)].index
    else:
        # Menggunakan z-score untuk mendeteksi outlier
        z_scores = np.abs((val_set[col] - val_set[col].mean()) / val_set[col].std())
        outlier_indices = val_set[z_scores > 3].index
    
    # Hitung persentase outlier
    outlier_percentage = len(outlier_indices) / len(val_set) * 100

    print (outlier_percentage)

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


### III. Remove Duplicates
Handling duplicate values is crucial because they can compromise data integrity, leading to inaccurate analysis and insights. Duplicate entries can bias machine learning models, causing overfitting and reducing their ability to generalize to new data. They also inflate the dataset size unnecessarily, increasing computational costs and processing times. Additionally, duplicates can distort statistical measures and lead to inconsistencies, ultimately affecting the reliability of data-driven decisions and reporting. Ensuring data quality by removing duplicates is essential for accurate, efficient, and consistent analysis.

In [17]:
jumlah_duplikat = train_set.duplicated().sum()
print(f"Jumlah baris duplikat: {jumlah_duplikat}")

Jumlah baris duplikat: 0


In [18]:
jumlah_duplikat = val_set.duplicated().sum()
print(f"Jumlah baris duplikat: {jumlah_duplikat}")

Jumlah baris duplikat: 0


In [19]:
# Write your code here
train_set = train_set.drop_duplicates()

In [20]:
# Write your code here
val_set = val_set.drop_duplicates()

In [21]:
jumlah_duplikat = train_set.duplicated().sum()
print(f"Jumlah baris duplikat: {jumlah_duplikat}")

Jumlah baris duplikat: 0


In [22]:
jumlah_duplikat = val_set.duplicated().sum()
print(f"Jumlah baris duplikat: {jumlah_duplikat}")

Jumlah baris duplikat: 0


### IV. Feature Engineering

**Feature engineering** involves creating new features (input variables) or transforming existing ones to improve the performance of machine learning models. Feature engineering aims to enhance the model's ability to learn patterns and make accurate predictions from the data. It's often said that "good features make good models."

1. **Feature Selection:** Feature engineering can involve selecting the most relevant and informative features from the dataset. Removing irrelevant or redundant features not only simplifies the model but also reduces the risk of overfitting.

2. **Creating New Features:** Sometimes, the existing features may not capture the underlying patterns effectively. In such cases, engineers create new features that provide additional information. For example:
   
   - **Polynomial Features:** Engineers may create new features by taking the square, cube, or other higher-order terms of existing numerical features. This can help capture nonlinear relationships.
   
   - **Interaction Features:** Interaction features are created by combining two or more existing features. For example, if you have features "length" and "width," you can create an "area" feature by multiplying them.

3. **Binning or Discretization:** Continuous numerical features can be divided into bins or categories. For instance, age values can be grouped into bins like "child," "adult," and "senior."

4. **Domain-Specific Feature Engineering:** Depending on the domain and problem, engineers may create domain-specific features. For example, in fraud detection, features related to transaction history and user behavior may be engineered to identify anomalies.

Feature engineering is both a creative and iterative process. It requires a deep understanding of the data, domain knowledge, and experimentation to determine which features will enhance the model's predictive power.

In [23]:
print(len(df.columns))

55


In [24]:
# Buat fitur baru dari karakteristik URL
if 'URL' in df.columns:
   # Hitung karakter spesial
   df['special_char_count'] = df['URL'].str.count(r'[^a-zA-Z0-9]').fillna(0)
   # Hitung digit
   df['digit_count'] = df['URL'].str.count(r'[0-9]').fillna(0)

# Buat fitur interaksi
if 'URLLength' in df.columns and 'DomainLength' in df.columns:
   df['url_domain_ratio'] = df['URLLength'] / df['DomainLength'].replace(0, 1)

# Buat fitur binary dari TLD
if 'TLD' in df.columns:
   common_tlds = ['com', 'org', 'net', 'edu']
   df['is_common_tld'] = df['TLD'].isin(common_tlds).astype(int)

# Buat aggregate features
security_cols = ['HasObfuscation', 'IsHTTPS', 'HasHiddenFields']
df['security_score'] = df[security_cols].sum(axis=1)

# Cek fitur baru
print("\nJumlah kolom setelah feature engineering:", len(df.columns))
print("\nBeberapa baris dengan fitur baru:")
print(df[['special_char_count', 'digit_count', 'url_domain_ratio',
          'is_common_tld', 'security_score']].head())


Jumlah kolom setelah feature engineering: 60

Beberapa baris dengan fitur baru:
   special_char_count  digit_count  url_domain_ratio  is_common_tld  \
0                   6            0          1.470588              1   
1                   5            2         19.000000              0   
2                   6            0          1.285714              0   
3                   8            0          0.000000              1   
4                   8            0          0.000000              1   

   security_score  
0             0.0  
1             0.0  
2             1.0  
3             0.0  
4             1.0  


## B. Data Preprocessing

**Data preprocessing** is a broader step that encompasses both data cleaning and additional transformations to make the data suitable for machine learning algorithms. Its primary goals are:

1. **Feature Scaling:** Ensure that numerical features have similar scales. Common techniques include Min-Max scaling (scaling to a specific range) or standardization (mean-centered, unit variance).

2. **Encoding Categorical Variables:** Machine learning models typically work with numerical data, so categorical variables need to be encoded. This can be done using one-hot encoding, label encoding, or more advanced methods like target encoding.

3. **Handling Imbalanced Classes:** If dealing with imbalanced classes in a binary classification task, apply techniques such as oversampling, undersampling, or using different evaluation metrics to address class imbalance.

4. **Dimensionality Reduction:** Reduce the number of features using techniques like Principal Component Analysis (PCA) or feature selection to simplify the model and potentially improve its performance.

5. **Normalization:** Normalize data to achieve a standard distribution. This is particularly important for algorithms that assume normally distributed data.

Drop Column that Useless

In [25]:
train_set = train_set.drop(columns=['Title', 'Domain', 'URL', 'TLD'])
val_set = val_set.drop(columns=['Title', 'Domain', 'URL', 'TLD'])

### Notes on Preprocessing processes

It is advised to create functions or classes that have the same/similar type of inputs and outputs, so you can add, remove, or swap the order of the processes easily. You can implement the functions or classes by yourself

or

use `sklearn` library. To create a new preprocessing component in `sklearn`, implement a corresponding class that includes:
1. Inheritance to `BaseEstimator` and `TransformerMixin`
2. The method `fit`
3. The method `transform`

In [26]:
# Example

# from sklearn.base import BaseEstimator, TransformerMixin

# class FeatureEncoder(BaseEstimator, TransformerMixin):

#     def fit(self, X, y=None):

#         # Fit the encoder here

#         return self

#     def transform(self, X):
#         X_encoded = X.copy()

#         # Encode the categorical variables here

#         return X_encoded

### I. Feature Scaling

**Feature scaling** is a preprocessing technique used in machine learning to standardize the range of independent variables or features of data. The primary goal of feature scaling is to ensure that all features contribute equally to the training process and that machine learning algorithms can work effectively with the data.

Here are the main reasons why feature scaling is important:

1. **Algorithm Sensitivity:** Many machine learning algorithms are sensitive to the scale of input features. If the scales of features are significantly different, some algorithms may perform poorly or take much longer to converge.

2. **Distance-Based Algorithms:** Algorithms that rely on distances or similarities between data points, such as k-nearest neighbors (KNN) and support vector machines (SVM), can be influenced by feature scales. Features with larger scales may dominate the distance calculations.

3. **Regularization:** Regularization techniques, like L1 (Lasso) and L2 (Ridge) regularization, add penalty terms based on feature coefficients. Scaling ensures that all features are treated equally in the regularization process.

Common methods for feature scaling include:

1. **Min-Max Scaling (Normalization):** This method scales features to a specific range, typically [0, 1]. It's done using the following formula:

   $$X' = \frac{X - X_{min}}{X_{max} - X_{min}}$$

   - Here, $X$ is the original feature value, $X_{min}$ is the minimum value of the feature, and $X_{max}$ is the maximum value of the feature.  
<br />
<br />
2. **Standardization (Z-score Scaling):** This method scales features to have a mean (average) of 0 and a standard deviation of 1. It's done using the following formula:

   $$X' = \frac{X - \mu}{\sigma}$$

   - $X$ is the original feature value, $\mu$ is the mean of the feature, and $\sigma$ is the standard deviation of the feature.  
<br />
<br />
3. **Robust Scaling:** Robust scaling is a method that scales features to the interquartile range (IQR) and is less affected by outliers. It's calculated as:

   $$X' = \frac{X - Q1}{Q3 - Q1}$$

   - $X$ is the original feature value, $Q1$ is the first quartile (25th percentile), and $Q3$ is the third quartile (75th percentile) of the feature.  
<br />
<br />
4. **Log Transformation:** In cases where data is highly skewed or has a heavy-tailed distribution, taking the logarithm of the feature values can help stabilize the variance and improve scaling.

The choice of scaling method depends on the characteristics of your data and the requirements of your machine learning algorithm. **Min-max scaling and standardization are the most commonly used techniques and work well for many datasets.**

Scaling should be applied separately to each training and test set to prevent data leakage from the test set into the training set. Additionally, **some algorithms may not require feature scaling, particularly tree-based models.**

In [27]:
# Identifikasi kolom numerik
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
X = df[numeric_cols].drop('label', axis=1)
y = df['label']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardization
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert kembali ke DataFrame
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

# Cek hasil

X_train_scaled.describe()

,id,URLLength,DomainLength,IsDomainIP,CharContinuationRate,TLDLegitimateProb,URLCharProb,TLDLength,NoOfSubDomain,HasObfuscation,...,NoOfImage,NoOfCSS,NoOfJS,NoOfSelfRef,NoOfEmptyRef,NoOfExternalRef,special_char_count,digit_count,url_domain_ratio,security_score
count,1.123230e+05,1.123230e+05,1.123230e+05,1.123230e+05,1.123230e+05,1.123230e+05,1.123230e+05,1.123230e+05,1.123230e+05,1.123230e+05,...,1.123230e+05,1.123230e+05,1.123230e+05,1.123230e+05,1.123230e+05,1.123230e+05,1.123230e+05,1.123230e+05,1.123230e+05,1.123230e+05
mean,-4.412307e-17,3.479238e-18,-6.009594e-17,2.909909e-18,-1.535673e-15,-2.391186e-17,1.509990e-15,5.161925e-16,2.119173e-17,9.362314e-18,...,-3.036426e-18,-2.214061e-19,3.096522e-17,2.941538e-18,1.182941e-17,1.075401e-17,2.720132e-17,-8.856243e-18,3.991635e-17,-1.184206e-16
std,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,...,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00
min,-1.728443e+00,-8.554171e-01,-9.221885e-01,-1.765498e-02,-7.023026e+00,-1.618404e+00,-9.126548e+00,-1.826148e+00,-1.218626e+00,-1.550600e-02,...,-3.165726e-01,-5.055096e-02,-5.231743e-01,-3.598095e-01,-1.254595e-01,-4.328311e-01,-7.987483e-01,-4.268815e-02,-4.743609e-01,-1.438838e+00
25%,-8.658087e-01,-8.554171e-01,-9.221885e-01,-1.765498e-02,3.995615e-01,-1.269101e+00,-1.668234e-01,3.975990e-01,-1.218626e+00,-1.550600e-02,...,-3.165726e-01,-5.055096e-02,-5.231743e-01,-3.598095e-01,-1.254595e-01,-1.521985e-01,-4.385573e-01,-4.268815e-02,-4.743609e-01,-6.376526e-03
50%,-2.088194e-03,1.812304e-01,-9.221885e-01,-1.765498e-02,3.995615e-01,6.658227e-01,9.464920e-02,3.975990e-01,3.143190e-01,-1.550600e-02,...,-2.094006e-01,-5.055096e-02,-4.107629e-01,-2.450191e-01,-1.254595e-01,-1.448134e-01,-4.385573e-01,-4.268815e-02,-4.089950e-01,-6.376526e-03
75%,8.669749e-01,3.971987e-01,8.664631e-01,-1.765498e-02,3.995615e-01,6.658227e-01,3.489423e-01,3.975990e-01,3.143190e-01,-1.550600e-02,...,4.066753e-02,5.170411e-03,2.637053e-01,1.306585e-01,-7.656581e-02,-1.374284e-01,6.420156e-01,-4.268815e-02,6.117195e-01,-6.376526e-03
max,1.732383e+00,1.742516e+02,9.433163e+00,5.664123e+01,3.995615e-01,6.658227e-01,4.546622e+00,1.818758e+01,6.446097e+00,6.449117e+01,...,1.063315e+02,3.326061e+02,1.584265e+02,1.384113e+02,2.388179e+02,2.027747e+02,2.625008e+02,2.976953e+02,1.829827e+02,2.858546e+00


### II. Feature Encoding

**Feature encoding**, also known as **categorical encoding**, is the process of converting categorical data (non-numeric data) into a numerical format so that it can be used as input for machine learning algorithms. Most machine learning models require numerical data for training and prediction, so feature encoding is a critical step in data preprocessing.

Categorical data can take various forms, including:

1. **Nominal Data:** Categories with no intrinsic order, like colors or country names.  

2. **Ordinal Data:** Categories with a meaningful order but not necessarily equidistant, like education levels (e.g., "high school," "bachelor's," "master's").

There are several common methods for encoding categorical data:

1. **Label Encoding:**

   - Label encoding assigns a unique integer to each category in a feature.
   - It's suitable for ordinal data where there's a clear order among categories.
   - For example, if you have an "education" feature with values "high school," "bachelor's," and "master's," you can encode them as 0, 1, and 2, respectively.
<br />
<br />
2. **One-Hot Encoding:**

   - One-hot encoding creates a binary (0 or 1) column for each category in a nominal feature.
   - It's suitable for nominal data where there's no inherent order among categories.
   - Each category becomes a new feature, and the presence (1) or absence (0) of a category is indicated for each row.
<br />
<br />
3. **Target Encoding (Mean Encoding):**

   - Target encoding replaces each category with the mean of the target variable for that category.
   - It's often used for classification problems.

In [28]:
# Write your code here
# Write your code here
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

# Inisialisasi OneHotEncoder
# The sparse argument is deprecated in newer versions.
# It's now implicitly handled based on other settings.
# You can remove it safely.
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore') # If you need to handle_unknown values

# Melakukan fit_transform pada kolom TLD
encoded_data = encoder.fit_transform(train_set[['TLD']])

# Mengubah hasil encoding menjadi DataFrame
encoded_train_set = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(['TLD']))

# Menggabungkan hasil encoding dengan DataFrame asli (opsional)
train_set = pd.concat([df, encoded_train_set], axis=1)

# Applying the encoding transformation to the validation set
encoded_val_data = encoder.transform(val_set[['TLD']])

# Creating a DataFrame with the encoded features for the validation set
encoded_val_set = pd.DataFrame(encoded_val_data, columns=encoder.get_feature_names_out(['TLD']))

# Concatenating the encoded features with the original validation set (optional)
val_set = pd.concat([val_set, encoded_val_set], axis=1)

KeyError: "None of [Index(['TLD'], dtype='object')] are in the [columns]"

In [ ]:
train_set

In [ ]:
val_set

### III. Handling Imbalanced Dataset

**Handling imbalanced datasets** is important because imbalanced data can lead to several issues that negatively impact the performance and reliability of machine learning models. Here are some key reasons:

1. **Biased Model Performance**:

 - Models trained on imbalanced data tend to be biased towards the majority class, leading to poor performance on the minority class. This can result in misleading accuracy metrics.

2. **Misleading Accuracy**:

 - High overall accuracy can be misleading in imbalanced datasets. For example, if 95% of the data belongs to one class, a model that always predicts the majority class will have 95% accuracy but will fail to identify the minority class.

3. **Poor Generalization**:

 - Models trained on imbalanced data may not generalize well to new, unseen data, especially if the minority class is underrepresented.


Some methods to handle imbalanced datasets:
1. **Resampling Methods**:

 - Oversampling: Increase the number of instances in the minority class by duplicating or generating synthetic samples (e.g., SMOTE).
 - Undersampling: Reduce the number of instances in the majority class to balance the dataset.

2. **Evaluation Metrics**:

 - Use appropriate evaluation metrics such as precision, recall, F1-score, ROC-AUC, and confusion matrix instead of accuracy to better assess model performance on imbalanced data.

3. **Algorithmic Approaches**:

 - Use algorithms that are designed to handle imbalanced data, such as decision trees, random forests, or ensemble methods.
 - Adjust class weights in algorithms to give more importance to the minority class.

In [ ]:
print("distribution:")
print(df['label'].value_counts())
print("\npercentage distribution:")
print(df['label'].value_counts(normalize=True) * 100)

In [ ]:
columns_to_drop_imbalanced = ['id', 'FILENAME', 'URL', 'Title', 'Domain']
X_imbalanced = df.drop(columns_to_drop_imbalanced + ['label'], axis=1)
X_imbalanced = pd.get_dummies(X_imbalanced, columns=['TLD'])
y_imbalanced = df['label']

print("features shape:", X_imbalanced.shape)
print("label shape:", y_imbalanced.shape)

X_train_imbalanced, X_test_imbalanced, y_train_imbalanced, y_test_imbalanced = train_test_split(
    X_imbalanced, 
    y_imbalanced, 
    test_size=0.3, 
    random_state=66
)

print("\ntraining set before resampling:")
print(y_train_imbalanced.value_counts())
print("\ntest set:")
print(y_test_imbalanced.value_counts())

from imblearn.combine import SMOTETomek
smote_tomek = SMOTETomek(random_state=66)
X_train_balanced, y_train_balanced = smote_tomek.fit_resample(X_train_imbalanced, y_train_imbalanced)

print("\ntraining set  after resampling:")
print(pd.Series(y_train_balanced).value_counts())

### V. Normalization

In [ ]:
train_set.describe()

In [ ]:
val_set.describe()

In [146]:
# numeric_cols = train_set.select_dtypes(include=['float64', 'int64']).columns

# min_max_scaler = MinMaxScaler()
# min_max_scaled = min_max_scaler.fit_transform(train_set[numeric_cols])

# train_set_min_max_scaled = train_set.copy()
# train_set_min_max_scaled[numeric_cols] = min_max_scaled

# print("\nMin-Max Scaled Data:")
# print(train_set_min_max_scaled)

# val_set_min_max_scaled = val_set.copy()
# val_set_min_max_scaled[numeric_cols] = min_max_scaler.transform(val_set[numeric_cols])

# print("\nMin-Max Scaled Data:")
# print(val_set_min_max_scaled)

# # Mengganti kolom numerik asli dengan kolom yang sudah di-scaling pada train_set
# train_set = train_set.drop(columns=numeric_cols)
# train_set = pd.concat([train_set, train_set_min_max_scaled[numeric_cols]], axis=1)

# # Mengganti kolom numerik asli dengan kolom yang sudah di-scaling pada val_set
# val_set = val_set.drop(columns=numeric_cols)
# val_set = pd.concat([val_set, val_set_min_max_scaled[numeric_cols]], axis=1)

# 3. Compile Preprocessing Pipeline

All of the preprocessing classes or functions defined earlier will be compiled in this step.

If you use sklearn to create preprocessing classes, you can list your preprocessing classes in the Pipeline object sequentially, and then fit and transform your data.

In [147]:
# from sklearn.pipeline import Pipeline

# # Note: You can add or delete preprocessing components from this pipeline

# pipe = Pipeline([("imputer", FeatureImputer()),
#                  ("featurecreator", FeatureCreator()),
#                  ("scaler", FeatureScaler()),
#                  ("encoder", FeatureEncoder())])

# train_set = pipe.fit_transform(train_set)
# val_set = pipe.transform(val_set)

In [148]:
# # Your code should work up until this point
# train_set = pipe.fit_transform(train_set)
# val_set = pipe.transform(val_set)

or create your own here

In [149]:
# Write your code here

# 4. Modeling and Validation

Modelling is the process of building your own machine learning models to solve specific problems, or in this assignment context, predicting the target feature `label`. Validation is the process of evaluating your trained model using the validation set or cross-validation method and providing some metrics that can help you decide what to do in the next iteration of development.

In [ ]:
# Identifikasi missing values
print("Jumlah missing values:")
print(val_set.isnull().sum())
print()

# Menghitung persentase missing values
missing_percent = (val_set.isnull().sum() / len(val_set)) * 100
print("Persentase missing values:")
print(missing_percent)

## A. KNN

In [150]:
X_train = train_set.drop('label', axis=1)
y_train = train_set['label']

# Splitting val_set into X_val and y_val
X_val = val_set.drop('label', axis=1)
y_val = val_set['label']

In [ ]:
import numpy as np

def euclidean_distance(x1, x2):
    return np.sqrt(np.sum((x1 - x2) ** 2))

def knn_predict(X_train, y_train, X_test, k):
    y_pred = []
    for x in X_test:
        distances = [euclidean_distance(x, x_train) for x_train in X_train]
        k_idx = np.argsort(distances)[:k]
        k_neighbor_labels = [y_train[i] for i in k_idx]
        most_common = max(set(k_neighbor_labels), key=k_neighbor_labels.count)
        y_pred.append(most_common)
    return np.array(y_pred)

# Assuming you have your training and validation data stored in X_train, y_train, X_val, y_val

# Set the value of k
k = 3

# Make predictions on the validation set
predictions = knn_predict(X_train, y_train, X_val, k)

# Calculate accuracy
accuracy = np.sum(predictions == y_val) / len(y_val)
print("Accuracy:", accuracy)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)  # You can adjust the value of n_neighbors
knn.fit(X_train, y_train)
knn_predictions = knn.predict(X_val)

## B. Naive Bayes

In [ ]:
nb = GaussianNB()
nb.fit(X_train, y_train)
nb_predictions = nb.predict(X_val)# Type your code here

In [ ]:
import numpy as np

def gaussian_naive_bayes(X_train, y_train, X_test):
    classes = np.unique(y_train)
    n_classes = len(classes)
    n_features = X_train.shape[1]
    
    # Calculate prior probabilities
    priors = np.zeros(n_classes)
    for i, c in enumerate(classes):
        priors[i] = np.sum(y_train == c) / len(y_train)
    
    # Calculate mean and variance for each feature per class
    means = np.zeros((n_classes, n_features))
    variances = np.zeros((n_classes, n_features))
    for i, c in enumerate(classes):
        X_c = X_train[y_train == c]
        means[i, :] = np.mean(X_c, axis=0)
        variances[i, :] = np.var(X_c, axis=0)
    
    # Make predictions for test set
    y_pred = []
    for x in X_test:
        posteriors = []
        for i, c in enumerate(classes):
            prior = np.log(priors[i])
            likelihood = np.sum(np.log(gaussian_pdf(x, means[i], variances[i])))
            posterior = prior + likelihood
            posteriors.append(posterior)
        y_pred.append(classes[np.argmax(posteriors)])
    
    return np.array(y_pred)

def gaussian_pdf(x, mean, var):
    return (1 / np.sqrt(2 * np.pi * var)) * np.exp(-((x - mean) ** 2) / (2 * var))

# Assuming you have your training and validation data stored in X_train, y_train, X_val, y_val

# Make predictions on the validation set
predictions = gaussian_naive_bayes(X_train, y_train, X_val)

# Calculate accuracy
accuracy = np.sum(predictions == y_val) / len(y_val)
print("Accuracy:", accuracy)

In [ ]:
# Evaluation Metrics
knn_accuracy = accuracy_score(y_val, knn_predictions)
knn_precision = precision_score(y_val, knn_predictions)
knn_recall = recall_score(y_val, knn_predictions)
knn_f1 = f1_score(y_val, knn_predictions)

nb_accuracy = accuracy_score(y_val, nb_predictions)
nb_precision = precision_score(y_val, nb_predictions)
nb_recall = recall_score(y_val, nb_predictions)
nb_f1 = f1_score(y_val, nb_predictions)

# Print the evaluation metrics
print("KNN Metrics:")
print("Accuracy:", knn_accuracy)
print("Precision:", knn_precision)
print("Recall:", knn_recall)
print("F1 Score:", knn_f1)

print("\nNaive Bayes Metrics:")
print("Accuracy:", nb_accuracy)
print("Precision:", nb_precision)
print("Recall:", nb_recall)
print("F1 Score:", nb_f1)

## C. Improvements (Optional)

- **Visualize the model evaluation result**

This will help you to understand the details more clearly about your model's performance. From the visualization, you can see clearly if your model is leaning towards a class than the others. (Hint: confusion matrix, ROC-AUC curve, etc.)

- **Explore the hyperparameters of your models**

Each models have their own hyperparameters. And each of the hyperparameter have different effects on the model behaviour. You can optimize the model performance by finding the good set of hyperparameters through a process called **hyperparameter tuning**. (Hint: Grid search, random search, bayesian optimization)

- **Cross-validation**

Cross-validation is a critical technique in machine learning and data science for evaluating and validating the performance of predictive models. It provides a more **robust** and **reliable** evaluation method compared to a hold-out (single train-test set) validation. Though, it requires more time and computing power because of how cross-validation works. (Hint: k-fold cross-validation, stratified k-fold cross-validation, etc.)

In [240]:
# Type your code here

## D. Submission
To predict the test set target feature and submit the results to the kaggle competition platform, do the following:
1. Create a new pipeline instance identical to the first in Data Preprocessing
2. With the pipeline, apply `fit_transform` to the original training set before splitting, then only apply `transform` to the test set.
3. Retrain the model on the preprocessed training set
4. Predict the test set
5. Make sure the submission contains the `id` and `label` column.

Note: Adjust step 1 and 2 to your implementation of the preprocessing step if you don't use pipeline API from `sklearn`.

In [241]:
# Type your code here

# 6. Error Analysis

Based on all the process you have done until the modeling and evaluation step, write an analysis to support each steps you have taken to solve this problem. Write the analysis using the markdown block. Some questions that may help you in writing the analysis:

- Does my model perform better in predicting one class than the other? If so, why is that?
- To each models I have tried, which performs the best and what could be the reason?
- Is it better for me to impute or drop the missing data? Why?
- Does feature scaling help improve my model performance?
- etc...

`Provide your analysis here`